注意力

In [2]:
import torch.nn as nn
import torch

In [3]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert(self.head_dim * heads== embed_size), "Embed size needs to be divisible by heads"
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N= query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(N, query_len, self.heads * self.head_dim)

        out = self.fc_out(out)
        return out






transformer块

In [4]:
class TransformerBlock(nn.Module):
    # 初始化TransformerBlock类，继承自nn.Module
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        # 调用父类的构造函数
        super(TransformerBlock, self).__init__()
        # 初始化自注意力机制模块
        self.attention = SelfAttention(embed_size, heads)
        # 初始化两个层归一化模块
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        # 初始化前馈神经网络模块
        self.feed_forward = nn.Sequential(
            # 第一层线性变换，将输入维度扩展到forward_expansion倍
            nn.Linear(embed_size, forward_expansion * embed_size),
            # ReLU激活函数
            nn.ReLU(),
            # 第二层线性变换，将维度还原回embed_size
            nn.Linear(forward_expansion * embed_size, embed_size)
        )

        # 初始化Dropout层，用于防止过拟合
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, value, key, query, mask):
        # 计算自注意力机制的输出
        attention = self.attention(value, key, query, mask)

        # 将自注意力输出与输入相加，并进行层归一化和Dropout
        x = self.dropout(self.norm1(attention + query))
        # 通过前馈神经网络处理
        forward = self.feed_forward(x)
        # 将前馈网络的输出与输入相加，并进行层归一化和Dropout
        out = self.dropout(self.norm2(forward + x))
        # 返回最终的输出
        return out

编码器

In [ ]:
class Encoder(nn.Module):
    def __init__(self,
                 src_vocab_size,
                    embed_size,
                    num_layers,
                    heads,
                    device,
                    forward_expansion,
                    dropout,
                    max_length):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [TransformerBlock(embed_size,
                              heads,
                              dropout=dropout,
                              forward_expansion=forward_expansion) 
                              for _ in range(num_layers)]
                              )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_lenth= x.shape
        positions = torch.arange(0, seq_lenth).expand(N, seq_lenth).to(self.device)
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out
        

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(embed_size, heads, dropout, forward_expansion)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


In [ ]:
class Decoder(nn.Module):
    def __init__(self, 
                 trg_vocab_size,   # 目标词汇表大小
                 embed_size,      # 嵌入层大小
                 num_layers,      # 解码器层数
                 heads,           # 多头注意力头数
                 forward_expansion,   # 前馈网络扩展因子
                 dropout,         # Dropout比率
                 device,          # 设备（CPU或GPU）
                 max_length):    # 最大序列长度
        super(Decoder, self).__init__()
        self.device = device
        # 词汇嵌入层，将目标词汇ID映射到嵌入向量
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        # 位置嵌入层，将位置索引映射到嵌入向量
        self.position_embedding = nn.Embedding(max_length, embed_size)

        # 解码器层列表，包含多个DecoderBlock
        self.layers = nn.ModuleList(
            [DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
             for _ in range(num_layers)]
        )
        # 输出层，将嵌入向量映射到目标词汇表大小
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        # Dropout层，用于防止过拟合
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape  # 获取输入的批次大小和序列长度
        # 生成位置索引，形状为[N, seq_length]
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        # 计算词嵌入和位置嵌入的和，并应用Dropout
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        # 通过每一层解码器块
        for layer in self.layers:
            # 输入x, 编码器输出enc_out, 编码器掩码src_mask, 解码器掩码trg_mask
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        # 通过输出层，得到最终的输出
        out = self.fc_out(x)
        return out


In [ ]:
class Transformer(nn.Module):
    def __init__(self,
                 src_vocab_size, 
                 trg_vocab_size,
                 src_pad_idx,
                 trg_pad_idx,
                 embed_size=256,
                 num_layers=6,
                 forward_expansion=4,
                 heads=8,
                 dropout=0,
                 device="cuda",
                 max_length=100
                 ):
        # 初始化父类
        super(Transformer, self).__init__()
        # 初始化编码器
        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length
        )

        # 初始化解码器
        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length
        )
        # 保存填充索引和设备信息
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        # 创建源序列的掩码，用于屏蔽填充部分
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        #(N, 1, 1, src_len)
        return src_mask.to(self.device)
    
    def make_trg_mask(self, trg):
        # 创建目标序列的掩码，用于屏蔽未来的信息
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )
        return trg_mask.to(self.device)
    
    def forward(self, src, trg):
        # 生成源序列和目标序列的掩码
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        # 通过编码器处理源序列
        enc_src = self.encoder(src, src_mask)
        # 通过解码器处理目标序列，并结合编码器的输出
        out = self.decoder(trg, enc_src, src_mask,  trg_mask)
        # 返回解码器的输出
        return out


In [ ]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(
        device
    )
    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

    src_pad_idx = 0
    trg_pad_idx = 0
    src_vocab_size = 10
    trg_vocab_size = 10
    model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(device)
    out = model(x, trg[:, :-1])
    print(out.shape)